In [19]:
import pandas as pd
import pickle
class AssociationRecommendationModel:
    def __init__(self, data, rules):
        self.data = data
        self.rules = rules

    def get_top_associated_categories(self, item_ids, n=3):
        """Get top associated categories for a list of item IDs."""
        top_associated_categories = set()

        for item_id in item_ids:
            # Retrieve item information
            item_info = self.data[self.data['id'] == item_id]
            if item_info.empty:
                print(f"Item with ID {item_id} not found in dataset.")
                continue

            # Construct item category combination
            item_category = f"{item_info.iloc[0]['main_category']} - {item_info.iloc[0]['sub_category']}"
            # Filter rules to find relevant antecedents
            relevant_rules = self.rules[self.rules['antecedents'].apply(lambda x: item_category in x)]
            # If no rules, continue
            if relevant_rules.empty:
                continue

            # Sort rules and collect associated categories
            relevant_rules = relevant_rules.sort_values(by='lift', ascending=False).head(n)
            for _, row in relevant_rules.iterrows():
                for category_set in [row['antecedents'], row['consequents']]:
                    if isinstance(category_set, str):  # Ensure it's a string
                        for category in category_set.split(", "):  # Split by delimiter
                            if category != item_category:  # Exclude the original category
                                top_associated_categories.add(category)
        # Return the top N categories as a list
        top_categories_list = list(top_associated_categories)[:n]
        return top_categories_list


    def get_top_rated_items(self, associated_categories, top_n=5):
        """Get top-rated items from the associated categories."""
        top_items = []
        for category in associated_categories:
            try:
                # Split the main and subcategories
                main_category, sub_category = category.split(" - ")
            except ValueError:
                print(f"Invalid category format: {category}")
                continue
            # Filter items within the given category
            category_items = self.data[
                (self.data['main_category'] == main_category) &
                (self.data['sub_category'] == sub_category)
            ]
            # Get top items based on ratings
            top_rated_items = category_items.sort_values(by='ratings', ascending=False).head(top_n)
            top_items.extend(top_rated_items['id'].tolist())
        return top_items

    def recommend(self, item_ids, associated_n=3, top_n=5):
        # Get top associated categories for the input items
        top_associated_categories = self.get_top_associated_categories(item_ids, n=associated_n)
        # Get highest-rated items from the associated categories
        top_rated_items = self.get_top_rated_items(top_associated_categories, top_n=top_n)
        return top_rated_items


    def save(self, file_path):
        with open(file_path, 'wb') as f:
            pickle.dump(self, f)
        print(f"Model saved to {file_path}")

    @staticmethod
    def load(file_path):
        with open(file_path, 'rb') as f:
            model = pickle.load(f)
        print(f"Model loaded from {file_path}")
        return model


In [23]:
if __name__ == "__main__":
    # Load datasets
    product = pd.read_csv('test_data.csv')
    rules = pd.read_csv('association_rules.csv')  # Ensure this file contains 'antecedents', 'consequents', and 'lift'

    # Initialize the model
    model = AssociationRecommendationModel(data=product, rules=rules)

    # Save the model to a Pickle file
    model.save('D:/RecommendModel/Saved_model/association_recommendation_model.pkl')

    loaded_model = AssociationRecommendationModel.load('D:/RecommendModel/Saved_model/association_recommendation_model.pkl')

    example_item_ids = ['app010812','mcl257235']
    recommendations = model.recommend(item_ids=example_item_ids, associated_n=3, top_n=5)
    print("Final Recommendations:", recommendations)

Model saved to D:/RecommendModel/Saved_model/association_recommendation_model.pkl
Model loaded from D:/RecommendModel/Saved_model/association_recommendation_model.pkl
Final Recommendations: ['mcl185969', 'mcl172290', 'mcl118125', 'mcl139203', 'mcl180466', 'app203321', 'app013021', 'app091999', 'app110901', 'app120588', 'app125574', 'app107571', 'app042768', 'app066402', 'app073406']
